<a href="https://colab.research.google.com/github/ooutaksaluk/MBT-analysis/blob/main/Main_Battle_Tank_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**Main Battle Tank analysis**



In [1]:
!pip install anvil-uplink opencv-python-headless numpy torch torchvision
!pip install ultralytics
!pip install torch torchvision
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


  Using cached ultralytics-8.3.92-py3-none-any.whl.metadata (35 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
Using cached ultralytics-8.3.92-py3-none-any.whl (949 kB)
Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)
Cloning into 'yolov5'...
remote: Enumerating objects: 17274, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 17274 (delta 1), reused 0 (delta 0), pack-reused 17271 (from 3)
Receiving objects: 100% (17274/17274), 16.13 MiB | 16.54 MiB/s, done.
Resolving deltas: 100% (11859/11859), done.
/content/yolov5


In [13]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')  # Mount Google Drive to access files

# กำหนดเส้นทางของโฟลเดอร์ Dataset ใน Google Drive
dataset_path = "/content/drive/My Drive/dataset_yolo"

# ข้อมูลที่ต้องการใส่ในไฟล์ dataset.yaml
dataset_yaml = """
train: /content/drive/My Drive/dataset_yolo/images/train
val: /content/drive/My Drive/dataset_yolo/images/val

nc: 2  # จำนวนคลาส
names: ["Sight", "Mantlet"]  # ชื่อคลาส
"""

# สร้างไฟล์ dataset.yaml ใน Google Drive
with open(f"{dataset_path}/dataset.yaml", "w") as f:
    f.write(dataset_yaml)

print("✅ ไฟล์ dataset.yaml ถูกสร้างเรียบร้อย!")

Mounted at /content/drive
✅ ไฟล์ dataset.yaml ถูกสร้างเรียบร้อย!


In [14]:
import os
import cv2

# กำหนดเส้นทางของภาพและ label
images_path = "/content/drive/My Drive/dataset_yolo/images/train"
labels_path = "/content/drive/My Drive/dataset_yolo/labels/train"

# ตรวจสอบว่าไฟล์ภาพอยู่ในเส้นทางหรือไม่
image_file = os.path.join(images_path, "S1.JPG")
if not os.path.exists(image_file):
    print(f"ไม่พบไฟล์ {image_file}")
else:
    print(f"ไฟล์ {image_file} พบในระบบ")

# ลองโหลดภาพ
image = cv2.imread(image_file)

if image is None:
    print(f"ไม่สามารถโหลดภาพ {image_file} ได้!")
else:
    print(f"โหลดภาพ {image_file} สำเร็จ")


ไฟล์ /content/drive/My Drive/dataset_yolo/images/train/S1.JPG พบในระบบ
โหลดภาพ /content/drive/My Drive/dataset_yolo/images/train/S1.JPG สำเร็จ


In [ ]:
# import cv2
# import os
# import random
# import numpy as np

# # กำหนดโฟลเดอร์ที่เก็บภาพและไฟล์ label
# images_path = "/content/drive/My Drive/dataset_yolo/images/train"
# labels_path = "/content/drive/My Drive/dataset_yolo/labels/train"

# # จำนวนที่ต้องการเพิ่ม
# num_augmented_images = 500

# # ฟังก์ชันสำหรับการเพิ่มข้อมูล (Data Augmentation)
# def augment_image(image, label_file):
#     # อ่านข้อมูล label
#     with open(label_file, 'r') as f:
#         labels = f.readlines()

#     # การหมุนภาพ
#     angle = random.randint(-30, 30)  # หมุนภาพระหว่าง -30 ถึง 30 องศา
#     M = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1)
#     rotated_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

#     # การพลิกภาพแนวนอน (Horizontal Flip)
#     if random.random() > 0.5:
#         rotated_image = cv2.flip(rotated_image, 1)

#     # การขยาย/ย่อภาพ (Scaling)
#     scale_factor = random.uniform(0.8, 1.2)
#     scaled_image = cv2.resize(rotated_image, None, fx=scale_factor, fy=scale_factor)

#     # ปรับแสงของภาพ
#     brightness = random.randint(-50, 50)
#     augmented_image = cv2.convertScaleAbs(scaled_image, alpha=1, beta=brightness)

#     # อัปเดตข้อมูล label หลังการหมุนหรือพลิกภาพ
#     augmented_labels = []
#     for label in labels:
#         parts = label.split()
#         class_id = int(parts[0])  # เลขคลาส
#         x_center = float(parts[1])  # ค่าพิกัด x_center
#         y_center = float(parts[2])  # ค่าพิกัด y_center
#         width = float(parts[3])  # ความกว้าง
#         height = float(parts[4])  # ความสูง

#         # คำนวณพิกัดใหม่สำหรับ bounding box (ถ้ามีการพลิกหรือหมุนภาพ)
#         if angle != 0:
#             # โค้ดสำหรับคำนวณการหมุน
#             pass  # จะใช้การคำนวณพิกัดใหม่จากมุมการหมุนที่กำหนด

#         if scale_factor != 1:
#             x_center *= scale_factor
#             y_center *= scale_factor
#             width *= scale_factor
#             height *= scale_factor

#         augmented_labels.append(f"{class_id} {x_center} {y_center} {width} {height}\n")

#     return augmented_image, augmented_labels

# # เริ่มการเพิ่มภาพใหม่สำหรับทั้งสองคลาส (Sight และ Mantlet)
# counter = 1  # ตัวแปรสำหรับนับจำนวนภาพที่เพิ่มขึ้น
# while counter <= num_augmented_images:
#     # สุ่มเลือกภาพจาก 'Sight' หรือ 'Mantlet'
#     for class_name in ["Sight", "Mantlet"]:
#         if counter > num_augmented_images:  # ถ้าเกินจำนวนที่ต้องการแล้วหยุด
#             break

#         # สุ่มเลือกภาพจากไฟล์ C1, C2,... หรือ M1, M2,...
#         for i in range(1, 46):  # มีภาพ 45 ภาพในแต่ละคลาส
#             # สร้างชื่อไฟล์ภาพ
#             image_file = os.path.join(images_path, f"{class_name[0]}{i}.JPG")
#             label_file = os.path.join(labels_path, f"{class_name[0]}{i}.txt")

#             # อ่านภาพ
#             image = cv2.imread(image_file)

#             # ตรวจสอบว่าอ่านภาพได้หรือไม่
#             if image is None:
#                 print(f"ไม่สามารถโหลดภาพ {image_file} ได้! ข้ามไฟล์นี้ไป.")
#                 continue  # ข้ามไฟล์นี้ไป

#             # เพิ่มข้อมูล (การหมุน, พลิก, ขยาย, ปรับแสง)
#             augmented_image, augmented_labels = augment_image(image, label_file)

#             # บันทึกภาพใหม่
#             augmented_image_file = os.path.join(images_path, f"{class_name[0]}{counter}_augmented.JPG")
#             cv2.imwrite(augmented_image_file, augmented_image)

#             # บันทึกไฟล์ label ใหม่
#             augmented_label_file = os.path.join(labels_path, f"{class_name[0]}{counter}_augmented.txt")
#             with open(augmented_label_file, 'w') as f:
#                 f.writelines(augmented_labels)

#             print(f"เพิ่มข้อมูลเสร็จแล้วสำหรับ: {augmented_image_file}")
#             counter += 1  # เพิ่มตัวนับภาพ

# print("เพิ่มข้อมูลครบ 500 ภาพแล้ว!")


เพิ่มข้อมูลเสร็จแล้วสำหรับ: /content/drive/My Drive/dataset_yolo/images/train/S1_augmented.JPG
เพิ่มข้อมูลเสร็จแล้วสำหรับ: /content/drive/My Drive/dataset_yolo/images/train/S2_augmented.JPG
เพิ่มข้อมูลเสร็จแล้วสำหรับ: /content/drive/My Drive/dataset_yolo/images/train/S3_augmented.JPG
เพิ่มข้อมูลเสร็จแล้วสำหรับ: /content/drive/My Drive/dataset_yolo/images/train/S4_augmented.JPG
เพิ่มข้อมูลเสร็จแล้วสำหรับ: /content/drive/My Drive/dataset_yolo/images/train/S5_augmented.JPG
เพิ่มข้อมูลเสร็จแล้วสำหรับ: /content/drive/My Drive/dataset_yolo/images/train/S6_augmented.JPG
เพิ่มข้อมูลเสร็จแล้วสำหรับ: /content/drive/My Drive/dataset_yolo/images/train/S7_augmented.JPG
เพิ่มข้อมูลเสร็จแล้วสำหรับ: /content/drive/My Drive/dataset_yolo/images/train/S8_augmented.JPG
เพิ่มข้อมูลเสร็จแล้วสำหรับ: /content/drive/My Drive/dataset_yolo/images/train/S9_augmented.JPG
เพิ่มข้อมูลเสร็จแล้วสำหรับ: /content/drive/My Drive/dataset_yolo/images/train/S10_augmented.JPG
เพิ่มข้อมูลเสร็จแล้วสำหรับ: /content/drive/My Dri

**Training Model**

In [24]:
import torch
print(torch.cuda.device_count())  # จำนวน GPU
print(torch.cuda.get_device_name(0))  # ชื่อ GPU
print(torch.cuda.memory_allocated())  # หน่วยความจำที่ใช้ไป
print(torch.cuda.memory_reserved())  # หน่วยความจำที่จองไว้


1
Tesla T4
0
0


In [30]:
!WANDB_DISABLED=true python train.py --img 640 --batch 16 --epochs 50 --data /content/drive/My Drive/dataset_yolo/dataset.yaml --weights yolov5s.pt --device 0


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-03-18 20:31:04.193991: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742329864.215025    6483 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742329864.221340    6483 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 3
wandb: Y




**Anvil**



In [ ]:
import anvil.server
import anvil.media
import torch
import numpy as np
import cv2
from PIL import Image
from torchvision import transforms

# เชื่อมต่อกับ Anvil
anvil.server.connect("server_LX2D76OV3RCR3XCSOMCFYQZL-JGQCERXRR5DWB5QH")

# โหลดโมเดล YOLOv5 (ใช้ YOLOv5m ที่มีความแม่นยำสูงกว่า YOLOv5s)
model = torch.hub.load('ultralytics/yolov5', 'yolov5m')  # ใช้โมเดล YOLOv5m

# กำหนดคลาสที่เราต้องการตรวจจับ (เช่น 'Sight' และ 'Mantlet')
target_classes = [0, 1]  # ตัวอย่าง class_id สำหรับ 'Sight' และ 'Mantlet', ให้ปรับตามที่คุณฝึก

@anvil.server.callable
def process_image(file):
    print("เริ่มประมวลผลรูปภาพ...")

    # โหลดรูปภาพจาก Anvil
    file_bytes = file.get_bytes()
    nparr = np.frombuffer(file_bytes, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    if img is None:
        print("ไม่สามารถโหลดรูปภาพได้!")
        return None

    print("โหลดภาพสำเร็จ กำลังประมวลผล...")

    # แปลงภาพเป็น RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # ใช้โมเดล YOLOv5 สำหรับการทำนาย
    results = model(img_rgb)

    # ดึงผลลัพธ์ (bounding boxes, labels, confidence scores)
    pred_boxes = results.xyxy[0].cpu().numpy()  # [x_min, y_min, x_max, y_max, confidence, class_id]
    pred_scores = pred_boxes[:, 4]  # confidence score
    pred_classes = pred_boxes[:, 5].astype(int)  # class_id

    # กำหนด threshold ความมั่นใจของผลลัพธ์ (เช่น 0.3)
    threshold = 0.3  # ลดค่า threshold เป็น 0.3 เพื่อจับ object ที่มี confidence น้อยลง

    # ตรวจสอบว่ามีผลลัพธ์ที่มี confidence สูงพอและตรงกับคลาสที่เราต้องการ
    found_objects = False
    for i in range(len(pred_scores)):
        if pred_scores[i] >= threshold and pred_classes[i] in target_classes:
            found_objects = True  # หากพบ bounding box สำหรับคลาสที่ต้องการ
            x_min, y_min, x_max, y_max = map(int, pred_boxes[i][:4])
            class_id = int(pred_classes[i])
            confidence = float(pred_scores[i])

            # วาดกรอบ Bounding Box
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(img, f"Class {class_id} ({confidence:.2f})",
                        (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 2)

    # หากไม่พบวัตถุใดๆ
    if not found_objects:
        print("ไม่พบ bounding box ในภาพที่ควรมี!")

    # ส่งผลลัพธ์กลับไปยัง Anvil
    _, img_encoded = cv2.imencode('.jpg', img)
    return anvil.BlobMedia("image/jpeg", img_encoded.tobytes())


# เริ่มเซิร์ฟเวอร์ Anvil
anvil.server.wait_forever()


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-3-18 Python-3.11.11 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


เริ่มประมวลผลรูปภาพ...
โหลดภาพสำเร็จ กำลังประมวลผล...


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


ไม่พบ bounding box ในภาพที่ควรมี!
เริ่มประมวลผลรูปภาพ...
โหลดภาพสำเร็จ กำลังประมวลผล...


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


ไม่พบ bounding box ในภาพที่ควรมี!


In [ ]:
_, img_encoded = cv2.imencode('.JPG', img)
    return anvil.BlobMedia("image/jpeg", img_encoded.tobytes())